In [ ]:
from typing import Optional

import sympy
from sympy import sympify
from sympy.parsing.latex import parse_latex

def latex_to_int_modded(text: str) -> Optional[int]:
    try:
        # lark backend due to antlr not working for some reason
        sympy_expr = parse_latex(text, backend="antlr")
        # sympy_expr = parse_latex(text, backend="lark")
        # convert expr that is perfecly castable to integer, e.g.: sqrt(4) == 2.0
        # but thanks to Rational, we can detect this is in fact an int (2)
        print(sympy_expr)
        result = sympy.Rational(sympy_expr)
        if result.is_integer:
            return result.numerator // result.denominator
        return None
    except Exception as e:
        print(e)
        return None

# \\boxed{4046 \\times 4045!!}
latex_to_int_modded("3!")

In [ ]:
from aimo2.parser.latex import MyLatexParser
from pathlib import Path


MyLatexParser(grammar_dir=Path("../aimo2/parser").resolve(), mod=1000, timeout=1.0)

In [ ]:
from lark import Lark
parser = MyLatexParser(Path("../aimo2/parser").resolve(), mod=1000, timeout=0.3)
ans = parser.latex_to_int_modded("11132141!")
ans

In [ ]:
from sympy import sympify


s = sympify(" 6 /6")
int(s)

In [ ]:
[a for a in dir(ans) if a.startswith("is_")]

In [ ]:
# sympy_expr = parse_latex("2!", backend="antlr")
# # sympy_expr = parse_latex(text, backend="lark")
# # convert expr that is perfecly castable to integer, e.g.: sqrt(4) == 2.0
# # but thanks to Rational, we can detect this is in fact an int (2)
# print(sympy_expr)
# result = sympy.Rational(sympy_expr)
# if result.is_integer:
#     return result.numerator // result.denominator
# return None

In [ ]:
# sympy_expr = parse_latex("5! mod 10", backend="antlr")
# int(sympy_expr)


a = sympify("4046 * 4045!")

In [ ]:
from typing import Optional

import sympy
from sympy.parsing.latex import parse_latex
from sympy import factorial, factorial2, S

def latex_to_int_modded(text: str) -> Optional[int]:
    try:
        # Parse LaTeX with lark backend
        sympy_expr = parse_latex(text, backend="lark")

        # Function to replace nested factorials (from !!)
        def replace_double_factorial(expr):
            if expr.func == factorial and expr.args[0].func == factorial:
                # Detected !! pattern, e.g., factorial(factorial(n))
                n = expr.args[0].args[0]  # Extract the base number
                return factorial2(n)
            return expr.func(*[replace_double_factorial(arg) for arg in expr.args])

        # Apply the replacement
        corrected_expr = replace_double_factorial(sympy_expr)

        # Convert to Rational to check if it's an integer
        result = sympy.Rational(corrected_expr)
        if result.is_integer:
            return result.numerator // result.denominator
        return None
    except Exception:
        return None

# Test cases
print(latex_to_int_modded(r"3!!"))      # Should return 3 (double factorial)
print(latex_to_int_modded(r"3!"))       # Should return 6 (single factorial)
print(latex_to_int_modded(r"\sqrt{4}")) # Should return 2

print(latex_to_int_modded(r"3.5"))      # Should return None (not an integer)

In [ ]:
from sympy.parsing.sympy_parser import parse_expr

parse_expr("3")

In [ ]:
import os
import tempfile
import lark
import sympy
from sympy.testing.pytest import raises
from sympy.parsing.latex.lark import (
    LarkLaTeXParser,
    TransformToSymPyExpr,
    parse_latex_lark,
)
from importlib.resources import files


# grammar_file = os.path.join(os.path.dirname(__file__), "../latex/lark/grammar/latex.lark")
grammar_file = str(files("sympy.parsing.latex.lark.grammar") / "latex.lark")
print(grammar_file)



def init_custom_parser(modification, transformer=None):
    with open(grammar_file, encoding="utf-8") as f:
        latex_grammar = f.read()

    latex_grammar += modification

    with tempfile.NamedTemporaryFile() as f:
        f.write(bytes(latex_grammar, encoding="utf8"))

        parser = LarkLaTeXParser(grammar_file=f.name, transformer=transformer)

    return parser


class CustomTransformer(TransformToSymPyExpr):
    def number(self, tokens):
        if "," in tokens[0]:
            # The Float constructor expects a dot as the decimal separator
            return sympy.core.numbers.Float(tokens[0].replace(",", "."))
        else:
            return sympy.core.numbers.Integer(tokens[0])


def test_custom1():
    # Removes the parser's ability to understand \cdot and \div.
    modification1 = """
%override DIV_SYMBOL: DIV
%override MUL_SYMBOL: MUL | CMD_TIMES
"""
    parser = init_custom_parser(modification1)
    with raises(lark.exceptions.UnexpectedCharacters):
        parser.doparse(r"a \cdot b")
        parser.doparse(r"x \div y")


def test_custom2():
    # Makes the parser parse commas as the decimal separator instead of dots
    with raises(lark.exceptions.UnexpectedCharacters):
        # Asserting that the default parser cannot parse numbers which have commas as
        # the decimal separator
        parse_latex_lark("100,1")
        parse_latex_lark("0,009")

    modification2 = r"""
%override number: /\d+(,\d*)?/
"""
    parser = init_custom_parser(modification2, CustomTransformer)
    parser.doparse("100,1")
    parser.doparse("0,009")
    parser.doparse("2,71828")
    parser.doparse("3,14159")


def test_custom3():
    # makes sympy recognize double factorial
    modification3 = """
DOUBLE_BANG: "@"
factorial2: _expression_func DOUBLE_BANG
%extend _function: factorial2
"""
    parser = init_custom_parser(modification3)
    print(parser.doparse("3@"))

test_custom3()

In [ ]:
from sympy.parsing.latex.lark import parse_latex_lark

In [ ]:
a = """
%e
"""
a

# manual Lark

In [ ]:
# make sympy latex parser but we swap the parser with ours, this is to allow quick modification
# TODO how does this one maps to sympy?


class MyTransformToSymPy(TransformToSymPyExpr):
    def factorial2(self, tokens):
        print(tokens)
        return sympy.factorial2(tokens[0])

parser = LarkLaTeXParser(grammar_file="modified.lark", transformer=MyTransformToSymPy)


a = parser.doparse("3!!".replace("!!", "@"))
a

In [ ]:
type(a)

In [ ]:
asdasdsad

In [ ]:
import lark
from pathlib import Path



root = Path("/home/evan/Documents/kaggle/kaggle-aimo2/.venv/lib/python3.10/site-packages/sympy/parsing/latex/lark/grammar/")


with open(root / "latex.lark", "r") as f:
    latex_grammar = f.read()

parser = lark.Lark(
    grammar=latex_grammar,
    source_path='/home/evan/Documents/kaggle/kaggle-aimo2/.venv/lib/python3.10/site-packages/sympy/parsing/latex/lark/grammar/',
    parser="earley",
    start="latex_string",
    lexer="auto",
    ambiguity="explicit",
    propagate_positions=False,
    maybe_placeholders=False,
    keep_all_tokens=True,
)

In [ ]:
from lark.load_grammar import load_grammar

In [ ]:
load_grammar